## Install packages

In [1]:
#changes
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Ditto
! pwd

import os,sys
nb_path = '/content/notebooks'
os.symlink('/content/gdrive/MyDrive/Ditto', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)

Mounted at /content/gdrive
/content/gdrive/MyDrive/Ditto
/content/gdrive/MyDrive/Ditto


In [ ]:
# clone the repo
#!git clone https://github.com/megagonlabs/ditto
#!git clone https://github.com/NVIDIA/apex


In [2]:
import nltk
nltk.download('stopwords')

%cd ditto/
# install requirements
!pip install -r requirements.txt
%cd ..
!pip install transformers
!pip install tensorboardX
!pip install jsonlines

%cd apex
!pip install -v --no-cache-dir ./
%cd ..
# some issue with colab
!pip install --upgrade "urllib3==1.25.4" awscli

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/content/gdrive/MyDrive/Ditto/ditto
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 62.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 89.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.8/679.8 KB 49.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 23.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Could not find a version that satisfies the requirement sentencepiece==0.1.85 (from versions: 0.0.0, 0.0.2, 0.0.3, 0.0.4, 0.0.5, 0.0.6, 0.0.7, 0.0.9, 0.1.0, 0.1.1, 0.1.2, 0.1.3, 0.1.83

## Install fp16 optimization

In [3]:
%cd ditto

/content/gdrive/MyDrive/Ditto/ditto


In [ ]:
!CUDA_VISIBLE_DEVICES=0 python train_ditto.py \
  --task Dirty/Walmart-Amazon \
  --batch_size 32 \
  --max_len 256 \
  --lr 3e-5 \
  --n_epochs 20 \
  --finetuning \
  --lm roberta \
  --fp16 \
  --da drop_col \
  --save_model


2023-03-27 06:22:18.686608: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-27 06:22:18.686807: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-27 06:22:18.686839: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.b

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python matcher.py \
  --task Dirty/Walmart-Amazon \
  --input_path data/er_magellan/Dirty/Walmart-Amazon/test.txt \
  --output_path output/output_erm_walmart-amazon_1.jsonl \
  --lm roberta \
  --max_len 256 \
  --use_gpu \
  --fp16 \
  --checkpoint_path checkpoints/

## Running the matcher

In [ ]:
#@title Default title text
# Step 1:
# train the wdc model with the save_model flag on
# it will produce two files *_dev.pt and *_test.pt which are the 
# best checkpoints on the dev set and the test set.
!CUDA_VISIBLE_DEVICES=0 python train_ditto.py \
  --task wdc_all_small \
  --batch_size 32 \
  --max_len 64 \
  --lr 3e-5 \
  --n_epochs 5 \
  --finetuning \
  --lm distilbert \
  --fp16 \
  --save_model \
  --da drop_col


In [ ]:

# Step 2: run the matcher
!CUDA_VISIBLE_DEVICES=0 python matcher.py \
  --task wdc_all_small \
  --input_path input/input_small.jsonl \
  --output_path output/output_small.jsonl \
  --lm distilbert \
  --max_len 64 \
  --use_gpu \
  --fp16 \
  --checkpoint_path checkpoints/

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python train_ditto.py \
  --task Structured/Beer \
  --batch_size 32 \
  --max_len 64 \
  --lr 3e-5 \
  --n_epochs 10 \
  --finetuning \
  --lm roberta \
  --fp16 \
  --save_model \
  --da drop_col \
  --balance

!CUDA_VISIBLE_DEVICES=0 python matcher.py \
  --task Structured/Beer \
  --input_path data/er_magellan/Structured/Beer/test.txt \
  --output_path output/output_small.jsonl \
  --lm roberta \
  --max_len 64 \
  --use_gpu \
  --fp16 \
  --checkpoint_path checkpoints/

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 98.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.4 MB/s eta 0:00:00


In [ ]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
special_tokens = tokenizer.special_tokens_map
print(special_tokens)

{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}


In [ ]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
text = "I love ice cream"
text_pair = "Ice cream is my favorite dessert"
encoded_input = tokenizer.encode(text, text_pair)
print(tokenizer.decode(encoded_input))

<s>I love ice cream</s></s>Ice cream is my favorite dessert</s>


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
text = "I love ice cream"
text_pair = "Ice cream is my favorite dessert"
encoded_input = tokenizer.encode(text, text_pair)
print(tokenizer.decode(encoded_input))

[CLS] I love ice cream [SEP] Ice cream is my favorite dessert [SEP]


In [ ]:
st1 = "COL name VAL lg 24 [SEP] COL name VAL lg 24"

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
special_tokens = tokenizer.special_tokens_map
print(special_tokens)

{'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}
